### load model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
from joblib import load

class model_prediction:
    def __init__(self, new_data_file, model_name, scaler_name, window_size=60):
        self._new_data_file = new_data_file
        self._model_name = model_name
        self._scaler_name = scaler_name
        self._window_size = window_size
        self._load_new_data()
        self._load_model()
        self._load_scaler()
        self._features = [['Open', 'High', 'Low', 'Close', 'Volume']]

    def _load_new_data(self):
        try:
            self._new_data = pd.read_csv(self._new_data_file, parse_dates=['Date'], index_col='Date')
        except Exception as e:
            print(f"Exception: {e}")

    def _load_model(self):
        try:
            self._model = load_model(self._model_name)
        except Exception as e:
            print(f"Exception: {e}")

    def _load_scaler(self):
        try:
            self.scaler = load(self._scaler_name)
        except Exception as e:
            print(f"Exception: {e}")

    def _prepare_predict_data(self):
        scaled_new_data = self._scaler.transform(self._new_data)
        self._prepared_data = np.array([scaled_new_data[-self._window_size:]])

    def _predict_new_data(self):
        self._model_predict_data = self._model.predict(self._predict_data)

    def _invert_normalized_data(self):
        # 反归一化需要重建完整的多变量矩阵（仅Close列有值，其他列置0）
        dummy_matrix = np.zeros((1, self._features.shape[1]))
        dummy_matrix[:, 3] = self._model_predict_data.flatten()  # 第4列是Close
        self._predicted_data = self._scaler.inverse_transform(dummy_matrix)[0, 3]

    def sigle_day_predict(self):
        self._prepare_predict_data()
        self._predict_new_data()
        self._invert_normalized_data()
        return self._predicted_data
        
    def multi_day_predict(self, days=5) -> list:
        self.predictions = []
        scaled_new_data = self._scaler.transform(self._new_data)
        initial_sequence = scaled_new_data[-self._window_size:]
        current_sequence = initial_sequence.copy()
        self._prepare_predict_data()

        for _ in range(days):
            self._model_predict_data = self._model.predict(current_sequence.reshape(1, self._window_size, -1))

            # update sequence: slide window
            new_row = current_sequence[-1].copy()
            new_row[3] = next_day_scaled[0][0]  # 更新Close列
            current_sequence = np.vstack([current_sequence[1:], new_row])
            self._invert_normalized_data()
            self.predictions.append(self._predicted_data)            
        return self.predictions
        


In [4]:
"""
predict.ipynb
predict with input data
1) load saved model:            stock = Stock_Model()
2) load saved model:           stock.load_model()
3) predict data:               stock.model_predict(data)
"""
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
from joblib import dump

model_name = "stock_model_name.keras" # or name.h5
# save scaler: dump(scaler, "scaler.joblib")
scaler_name = "scaler_stock_name.joblib"
# file name of saved features (use json file)
feature_file_name = "features_stock_name.json"
# load features
features = load_features(feature_file_name)
# input test data
input_data = None
model = load_model(model_name)
## scaler must be the that in the model training
scaler = load(scaler_name)

$IFX.DE: possibly delisted; no price data found  (period=10y)


### nomalize input data (here, the training scaler must be used！）
#### # Note: use "transform" instead of "fit_transform"

In [ ]:
## new_features = [['Open', 'High', 'Low', 'Close', 'Volume']]
scaled_data = scaler.transform(features)

### prepare to predict data

In [ ]:
window_size = 60
prepared_data = np.array([scaled_data[-window_size:]])

### predict data

In [ ]:
predict_scaled_data = model.predict(prepared_data)

### invert normalized data

In [ ]:
def _invert_normalized_data(self):
    # 反归一化需要重建完整的多变量矩阵（仅Close列有值，其他列置0）
    dummy_matrix = np.zeros((len(self._Y_predict), self.features.shape[1]))
    dummy_matrix[:, 3] = self._Y_predict.flatten()  # 第4列是Close
    self._Y_pred_actual = self._scaler.inverse_transform(dummy_matrix)[:, 3]
    return dummy_matrix

### visual result

In [ ]:
def visual_predict_result(predict_prices, actual_prices=None):
    plt.figure(figsize=(12, 6))
    plt.plot(predict_prices, marker='o', label='Predicted Prices')
    if actual_prices:
        plt.plot(actual_prices, marker='x', label='Actual Prices')
    plt.title(f'Next {future_days} Days Price Prediction')
    plt.xlabel('Days into Future')
    plt.ylabel('Price')
    plt.legend()
    plt.show()